In [1]:
# So far, it might seem that we got away with being sloppy in setting up our networks. Specifically, we did the following unintuitive things, which might not seem like they should work:

# 1. We defined the network architectures without specifying the input dimensionality.

# 2. We added layers without specifying the output dimension of the previous layer.

# 3. We even “initialized” these parameters before providing enough information to determine how many parameters our models should contain.

import torch
from torch import nn
from d2l import torch as d2l

In [2]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

/data2/miniconda3/envs/d2l-pytorch/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [3]:
# Consequently the framework has not yet initialized any parameters. We confirm by attempting to access the parameters below.
net[0].weight

<UninitializedParameter>

In [4]:
X = torch.rand(2, 20)
net(X)

net[0].weight.shape

torch.Size([256, 20])

In [5]:
net[0].weight

Parameter containing:
tensor([[ 0.0756,  0.0736, -0.1443,  ..., -0.1234,  0.1793, -0.0511],
        [ 0.1240,  0.1129,  0.0682,  ..., -0.1935, -0.1285, -0.1907],
        [-0.0296, -0.0375, -0.1536,  ..., -0.0410,  0.2041, -0.1730],
        ...,
        [ 0.1042,  0.0755, -0.1371,  ...,  0.1648, -0.1610,  0.1607],
        [-0.0472, -0.0290, -0.0658,  ..., -0.0888, -0.1265,  0.1876],
        [ 0.0488,  0.1951, -0.1470,  ..., -0.1036,  0.1775,  0.1600]],
       requires_grad=True)

In [6]:
@d2l.add_to_class(d2l.Module)  # @save
def apply_init(self, inputs, init=None):
    self.forward(*inputs)
    if init is not None:
        self.net.apply(init)

1. If you specify the input dimensions to the first layer but not to subsequent layers:

In this case, the first layer will be initialized immediately, as it has enough information to determine the input and output dimensions. However, the subsequent layers won't be initialized immediately, as they don't have enough information about their input dimensions. In deep learning frameworks like PyTorch and TensorFlow, you might encounter errors or warnings when you try to access or use uninitialized layers.

2. If you specify mismatching dimensions:

If you specify mismatching dimensions between layers, you will likely encounter errors during the forward pass of the network. The errors will occur when the output of one layer is passed as input to the next layer, and their dimensions do not match. In this case, you need to fix the dimensions to ensure that the output of one layer matches the input of the next layer.

3. If you have input of varying dimensionality:

If you have input of varying dimensionality, you can use parameter tying to share the same set of weights across different input dimensions. This allows your model to handle varying input dimensions while keeping the number of parameters manageable. Here are some approaches to handle varying input dimensions:

- Use fully convolutional layers: Convolutional layers can handle varying input dimensions as long as the input has the same number of channels as specified in the layer. The output dimensions will also vary accordingly, but the weights remain the same.

- Use adaptive pooling layers: Adaptive pooling layers, such as `nn.AdaptiveAvgPool2d` and `nn.AdaptiveMaxPool2d` in PyTorch, allow you to specify the output dimensions, and they will automatically adjust the pooling operation to match the input dimensions. This can help you achieve a fixed output size regardless of the input size.

- Use recurrent layers (RNN, LSTM, GRU): Recurrent layers can handle sequences of varying lengths. You can either pad the sequences to a fixed length or pack the sequences using `torch.nn.utils.rnn.pack_padded_sequence` and `torch.nn.utils.rnn.pad_packed_sequence` in PyTorch to handle varying lengths efficiently.

- Flatten and reshape the input: If the varying dimensionality is in the spatial dimensions (e.g., height and width), you can flatten the input and then reshape it to a fixed size before passing it to the subsequent layers. This can be done using `nn.Flatten` and `nn.Linear` layers in PyTorch, or by using `reshape` and `dense` layers in TensorFlow.

Remember that when using parameter tying or shared weights, it's important to ensure that the shared weights make sense for the varying input dimensions and that the model can still learn meaningful features from the input data.

在深度学习中，"parameter tying"（参数绑定）是指在模型的不同部分共享相同的权重。这意味着这些部分在训练和推理过程中使用相同的参数。参数绑定可以减少模型的总参数数量，节省内存和计算资源，并有助于防止过拟合。

参数绑定在某些场景下非常有用，例如：

当模型的多个部分执行相同的操作时，例如在自然语言处理中的词嵌入层，输入和输出嵌入层可以共享相同的权重矩阵。
当处理具有不同尺寸的输入数据时，例如在卷积神经网络中，卷积层可以在不同的输入尺寸上共享相同的卷积核。
在循环神经网络（RNN）中，相同的权重在整个序列的各个时间步上共享，以学习序列中的模式。
然而，需要注意的是，参数绑定并非总是最佳方法。在某些情况下，共享参数可能会限制模型学习不同特征的能力。因此，在决定是否使用参数绑定时，需要仔细考虑问题和模型的结构。